In [123]:
# These settings automatically re-render the widget when the JavaScipt file has changed

%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [124]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle, IntSlider, Box, jslink, HTML
from nzshm_rupture_widget import rupture_map, MapWidget, SliderWidget, FullScreenWidget, HomeWidget, legend, slider
import json

In [125]:
with open('small_styled_crustal.geojson') as json_file:
    styled_crustal = json.load(json_file)
    json_file.close()

polygon = {"type":"FeatureCollection","features": [{
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.82, 9.27, 1],
                [31.82, 5.64, 1],
                [34.87, 5.64, 1],
                [34.87, 9.27, 1]
            ]
        ],
        "type": "Polygon"
    }
}]}

color_map = {0.001: '#fcffa4',
 0.0005: '#fbbe23',
 0.0002: '#ea632a',
 0.0001: '#bc3754',
 5e-05: '#7f1e6c',
 2e-05: '#2b0b57',
 1e-05: '#000004'}


In [132]:
map = MapWidget(data=[styled_crustal, polygon], layout=Layout(grid_area='1 / 1 / -1 / -1'))
sliderWidget = slider(map)
fullscreen = FullScreenWidget()
homeWidget = HomeWidget(map)
legend_widget = legend("The Legend", color_map)
legend_widget.style={"width":"80px"}

html = HTML("<b>Section Detail</b><br/><p>hover over fault sections for more details.</p>", style={"background":"white", "border-radius":"5px"})
html.add_class("mapHtmlWidget")
def on_hover(msg):
    properties = msg["properties"]
    if properties is not None:
        value = f"<b>{properties['FaultName']}</b>"
        value += "<br />"
        value += f"Dip: {properties['DipDeg']}</br>"
        value += f"Rake: {properties['Rake']}</br>"
        value += f"Lower depth: {round(properties['LowDepth'],3)}</br>"    
        value += f"Participation rate: {properties['annual_rate']:.2E}</br>" 
        html.value = value
    else:
        html.value = json.dumps(properties)

map.on_hover(on_hover)
map.hover_style = {"fillColor":"red", "fillOpacity":"0.6", "color":"green"}

int_slider = IntSlider(min=0, max=len(map.data)-1, value=0)
jslink((map, "selection"), (int_slider, "value"))

right_bar = Box([html, sliderWidget])
right_bar.add_class('mapRightBar')
right_bar.add_class('mapBottomRight')

left_bar = Box([homeWidget, fullscreen, legend_widget])
left_bar.add_class('mapLeftBar')
left_bar.add_class('mapTopLeft')

gridBox = GridBox(children=[map, right_bar, left_bar],
        layout=Layout(
            width='100%',
            height='400px',
            grid_template_rows='50px auto 50px',
            grid_template_columns='100px auto 300px')
       )
gridBox.add_class("fullScreenTarget")

gridBox


GridBox(children=(MapWidget(data=[{'type': 'FeatureCollection', 'features': [{'id': '216', 'type': 'Feature', …

In [127]:
help(right_bar)

Help on Box in module ipywidgets.widgets.widget_box object:

class Box(ipywidgets.widgets.domwidget.DOMWidget, ipywidgets.widgets.widget_core.CoreWidget)
 |  Box(children=(), **kwargs)
 |  
 |  Displays multiple widgets in a group.
 |  
 |  The widgets are laid out horizontally.
 |  
 |  Parameters
 |  ----------
 |  children: iterable of Widget instances
 |      list of widgets to display
 |  
 |  box_style: str
 |      one of 'success', 'info', 'warning' or 'danger', or ''.
 |      Applies a predefined style to the box. Defaults to '',
 |      which applies no pre-defined style.
 |  
 |  Examples
 |  --------
 |  >>> import ipywidgets as widgets
 |  >>> title_widget = widgets.HTML('<em>Box Example</em>')
 |  >>> slider = widgets.IntSlider()
 |  >>> widgets.Box([title_widget, slider])
 |  
 |  Method resolution order:
 |      Box
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget_core.CoreWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.